# 5.1 将函数视作对象

In [2]:
def factorial(n):
    '''return n!'''
    return 1 if n <2 else n*factorial(n-1)

factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'return n!'

In [4]:
# factorial 是 function 类的实例
type(factorial)

function

In [5]:
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [7]:
# map 函数返回一个可迭代对象，里面的元素是把第一个参数（一个函数）应用到第二个参数（一个可迭代对象，此处是 range(11)）中各个元素上得到的结果
map(factorial,range(11))

In [8]:
list(map(factorial,range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

# 5.2 高阶函数

高阶函数：接受函数为参数，或者把函数作为结果返回的函数  
目前常用的高阶函数有 map、filter、reduce 和 apply(已移除)  
同时 map 和 filter 已经被列表推导或生成器表达式所替代

In [ ]:
def factorial(n):
    '''return n!'''
    return 1 if n <2 else n*factorial(n-1)

fact = factorial

list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [ ]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [ ]:
list(map(fact, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [ ]:
[fact(n) for n in range(6) if n % 2]

[1, 6, 120]

In [ ]:
# 归约函数 reduce 和 sum
from functools import reduce
from operator import add
reduce(add, range(100))

4950

In [ ]:
sum(range(100))

4950

all 和 any 也是内置的归约函数
* all(interable) 如果 interable 的每个元素都是真值， 返回 True；all([]) 返回 True
* any(interable) 只要 interable 的元素中有真值， 就返回 True；any([]) 返回False
* 可以理解为 all 默认为 Ture 遇到为假则返回 False
* 而 any 默认为 False 遇到真则返回 True

In [ ]:
all([])

True

In [ ]:
bool([])

False

# 5.3 匿名函数

In [ ]:
# 使用 lambda 表达式反转拼写，然后依次给单词排序
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key = lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

# 5.4 可调用对象

使用内置的 callable() 函数可以判断对象能否调用
Python 数据模型文档列出了 7 种可调用对象
* 用户定义的函数    使用 def 语句或 lambda 表达式创建
* 内置函数          使用 C 语言 (CPython) 实现的函数，如 len 或 time.strftime
* 内置方法          使用 C 语言实现的方法，如 dict.get
* 方法              在类的定义体中定义的函数
* 类                调用类时会运行类的 \_\_new__ 方法创建一个实例，然后运行 \_\_init__ 方法，初始化实例，最后把实力返回调用方
* 类的实例          如果定义了 \_\_call__ 方法，那么它的实例可以作为函数使用
* 生成器函数        使用 yield 关键字的函数或方法。调用生成器函数返回的是生成器对象

用户定义的可调用类型  
只需实现实例方法 \_\_call__ 任何 Python 对象都可以表现的像函数 

In [ ]:
# 可调用的 BingoCage 从打乱的列表中取出一个元素
import random

class BingoCage:
    
    def __init__(self, items) -> None:
        self._items = list(items)
        random.shuffle(self._items) # 将列表中所有元素打乱
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [ ]:
bingo = BingoCage(range(3))
bingo.pick()

0

In [ ]:
bingo()

LookupError: pick from empty BingoCage

# 5.6 函数内省

In [ ]:
# 列出常规对象没有而函数有的属性
class C: pass
obj = C()
def func(): pass
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

# 5.7 从定位参数到仅限关键字参数

In [ ]:
# 一个生成 HTML 标签的函数
def tag(name, *content, cls=None, **attrs):
    '''生成一个或多个 HTML 标签'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)


In [ ]:
tag('br')

'<br />'

In [ ]:
# 第一个参数后任意个参数会被 *content 捕获，存入一个元组
tag('p', 'hello')

'<p>hello</p>'

In [ ]:
# tag 函数签名中没有明确指定名称的关键字参数会被 **attrs 捕获，存入一个字典
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [ ]:
# cls 只能作为关键字参数传入
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [ ]:
tag(content="testing", name="img")

'<img content="testing" />'

In [ ]:
my_tag = {'name': 'img', 'title': 'Sunset', 'src': 'Sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class="framed" src="Sunset.jpg" title="Sunset" />'

In [ ]:
# 对于仅限关键字参数，应当将他们放于带 * 的参数后面，如果不想支持数量不定的定位参数，但是想支持仅限关键字参数，可在签名中放一个 * 
# 同时仅限关键字参数不一定要有默认值，可以如下 b 一样强制传入实参
def f(a, *, b):
    return a,b
f(1,b=2)

(1, 2)

# 5.8 获取关于参数的信息

In [ ]:
# 在指定长度附近截取字符串的函数
def clip(text, max_len=80):
    """在 max_len 前面或后面的第一个空格处截断文本"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.find(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

参数名称在 \_\_code__.co_varnames 中，不过里面还有函数定义体中创建的局部变量。  
因此，参数名为前 N 个字符串，而 N 的值由 \_\_code__.co_argcount 确定。  
这里不包含前缀为 * 或 ** 的变长参数。  
参数的默认值只能通过他们在 \_\_defaults__ 元组中的位置确定。因此要从后往前扫描才能把参数和默认值对应起来  
在这个示例中 clip 函数有两个参数，text 和 max_len，其中一i给有默认值，为 80，因此它必然属于最后一个参数

In [ ]:
# 提取关于函数参数的信息
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)

(80,)
<code object clip at 0x000002B2CBBA6C90, file "C:\Users\16052\AppData\Local\Temp/ipykernel_7676/1361252703.py", line 2>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [ ]:
# 默认值与参数名顺序相反并不容易使用，但我们有 inspect 模块
# 提取函数的签名
from inspect import signature
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [ ]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


inspect.Parameter.kind 的属性可以是以下五个值之一：
* POSITIONAL_OR_KEYWORD 可以通过定位参数和关键字参数传入的形参
* VAR_POSITIONAL        定位参数元组
* VAR_KEYWORD           关键字参数字典
* KEYWORD_ONLY          仅限关键字参数
* POSITIONAL_ONLY       仅限定位参数(目前 Python 声明函数的语法不支持，但有些使用 C 语言实现且不接受关键字参数的函数 如：divmod 支持)
除此之外 inspect.Parameter 还有一个 annotation (注解) 属性  
  
inspect.Signature 对象有个 bind 方法，可以把任意个参数绑定到签名中的形参上，所用的规则与实参到形参的匹配方式一样，框架可以使用这个方法在真则调用参数前验证参数

In [ ]:
import inspect

def tag(name, *content, cls=None, **attrs):
    '''生成一个或多个 HTML 标签'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

sig = inspect.signature(tag)    # 获取 tag 函数的签名
my_tag = {'name': 'img', 'title': 'Sunset', 'src': 'Sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag) # 把一个字典参数传给 .bind() 方法
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset', 'src': 'Sunset.jpg'})>

In [ ]:
for name,value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset', 'src': 'Sunset.jpg'}


In [ ]:
del my_tag['name']  # 删除必须指定的参数

KeyError: 'name'

In [ ]:
bound_args = sig.bind(**my_tag) # 此时再次绑定抛出错误，缺少 name 参数

TypeError: missing a required argument: 'name'

# 5.9 函数注解

In [ ]:
# 有注解的 clip 函数 注意第一行参数后 : 内容
def clip(text : str, max_len:'int > 0'=80) -> str:
    """在 max_len 前面或后面的第一个空格处截断文本"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.find(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [ ]:
clip.__annotations__    # 获取注解

{'text': str, 'max_len': 'int > 0', 'return': str}

In [ ]:
# 从函数签名中提取注解
from inspect import signature
sig = signature(clip)
sig.return_annotation

str

In [ ]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


# 5.10 支持函数式编程的包

##### operator 模块

In [ ]:
# 使用 reduce 函数和一个匿名函数计算阶乘
from functools import reduce
def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

In [ ]:
# 使用 reduce 和 operator.mul 计算阶乘
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

operator 模块还有一类函数，能替代从序列中取出元素或读取对象属性的 lanbda 表达式：itemgetter 和 attrgetter 会自行构建函数

In [ ]:
# 按照第 2 个字段顺序打印城市信息
# itergetter(1) 的作用等同于 lambda fields: fields[1]
metro_data = [
    ('Tokyo', 'JP', 36.933, (1, 2)),
    ('Delhi NCR', 'IN', 21.935, (1, 2)),
    ('Mexico City', 'MX', 20.142, (1, 2)),
    ('Sao Paulo', 'BR', 19.649, (1, 2)),
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (1, 2))
('Delhi NCR', 'IN', 21.935, (1, 2))
('Tokyo', 'JP', 36.933, (1, 2))
('Mexico City', 'MX', 20.142, (1, 2))


In [ ]:
# 如果把多个参数给 itemgetter 它构建的函数会返回提取的值构成的元组
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('BR', 'Sao Paulo')


In [ ]:
# 使用 attrgetter
from collections import namedtuple
Latlong = namedtuple('Latlong', 'lat long')
Metropoils = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropoils(name, cc, pop, Latlong(lat, long)) for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=Latlong(lat=1, long=2))

In [ ]:
metro_areas[0].coord.lat

1

In [ ]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key = attrgetter('coord.lat')):
    print(name_lat(city))

('Tokyo', 1)
('Delhi NCR', 1)
('Mexico City', 1)
('Sao Paulo', 1)


In [ ]:
# operator.methodcaller 它的作用与 attrgetter 类似，它会自行创建函数。methodcaller 创建的函数会在对象上调用参数指定的方法
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper') # s.upper()
upcase(s)

'THE TIME HAS COME'

In [ ]:
hiphenate = methodcaller('replace', ' ', '-') # s.replace(' ', '-')
hiphenate(s)

'The-time-has-come'

## 5.10.2 使用 functools.partial 冻结参数

functools.partial 这个高阶函数用于部分应用一个函数。部分应用是指，给予一个函数创建一个新的可调用对象，把原函数的某些参数固定。使用这个函数可以把接受一个或多个参数的函数改编成需要回调的 API，这样参数更少。

In [ ]:
from operator import mul
from functools import partial
triple = partial(mul, 3)    # 使用 mul 创建 triple 函数，将第一个定位参数定位 3
triple(7)

21

In [ ]:
list(map(triple, range(1,10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [ ]:
# 对 tag 进行参数冻结
def tag(name, *content, cls=None, **attrs):
    '''生成一个或多个 HTML 标签'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

picture = partial(tag, 'img', cls='pic-frame')
picture(src = 'a.png')

'<img class="pic-frame" src="a.png" />'

In [ ]:
picture

functools.partial(<function tag at 0x0000022E5911B8B0>, 'img', cls='pic-frame')

In [ ]:
# functools 对象提供了访问原函数和固定参数的属性
print(picture.func)
print(picture.args)
print(picture.keywords)

<function tag at 0x0000022E5911B8B0>
('img',)
{'cls': 'pic-frame'}
